In [1]:
import aisuite as ai
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
client = ai.Client()
model = ["openai:gpt-4o", "openai:gpt-4o-mini"]

load_dotenv() 

True

## Step 1: Analyze Prompt & Topic Recommendations

In [3]:
user_input = input("What do you want to write about? ")

messages = [
    {"role": "system", "content": "You are a senior content planner. You provide diverse blog topics based on users need and topics."},
    {"role": "user", "content": f"Based on the user's query: {user_input}, provide 5 compelling blog topics."},
]

response = client.chat.completions.create(
    model=model[1],
    messages=messages,
    temperature=0.75
)
print(response.choices[0].message.content)

Here are five compelling blog topics centered around digital nomad trends:

1. **"The Rise of the Digital Nomad: How Remote Work is Reshaping Global Employment Trends"**  
   Explore the factors contributing to the growth of the digital nomad lifestyle, including technological advancements, shifting corporate policies, and the desire for work-life balance.

2. **"Top Destinations for Digital Nomads in 2024: Where to Work and Play"**  
   A comprehensive guide to the best cities and countries for digital nomads in the coming year, focusing on cost of living, coworking spaces, community, and local amenities.

3. **"Navigating Digital Nomadism: Essential Tools and Apps for a Seamless Remote Work Experience"**  
   Highlight the must-have tools and applications that help digital nomads manage their work, stay organized, and connect with other remote workers around the globe.

4. **"From Side Hustle to Full-Time: Transitioning to a Digital Nomad Lifestyle"**  
   Provide practical tips and 

## Step 2: Confirm and Research Topic
https://serpapi.com/organic-results

In [25]:
# topic_query = input("What is the final topic you want to write about? ")
topic_query = "Top Digital Nomads Destinations in Asia"

In [26]:
base_url = "https://serpapi.com/search.json"
params = {
    "q": topic_query,
    "hl": "en",
    "gl": "us",
    "google_domain": "google.com",
    "api_key": os.getenv("SERPAPI_KEY")
}

# Make the API request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    results = response.json()
    # Process the results as needed
    print("Search results retrieved successfully.")
else:
    print(f"Failed to retrieve search results. Status code: {response.status_code}")

Search results retrieved successfully.


In [27]:
# Create a list to store the results
data = []

# Iterate over the results and append to the list
for result in results.get('organic_results', []):
    position = result.get('position')
    link = result.get('link')
    title = result.get('title')
    data.append({'Position': position, 'Link': link, 'Title': title})

# Convert the list to a DataFrame
df_results = pd.DataFrame(data)

df_results

,Position,Link,Title
0,1,https://www.reddit.com/r/digitalnomad/comments...,Which country / place in Asia is the best for ...
1,2,https://www.lostcoconuts.com/best-destinations...,12 best destinations in Southeast Asia for dig...
2,3,https://nomadsembassy.com/best-digital-nomad-c...,12 Best Digital Nomad Cities in Asia [2024 Edi...
3,4,https://www.thedigitalnomad.asia/,Digital Nomad Asia
4,5,https://www.goatsontheroad.com/digital-nomad-c...,7 Best Digital Nomad Cities in Asia
5,6,https://www.youtube.com/watch?v=IHUXp49Xlzg,5 INSANELY CHEAP Destinations for Digital Noma...
6,7,https://www.creimermanlaw.com/post/top-southea...,Top Southeast Asian Destinations for Expats an...


## Step 3: Analyze SEMRush Data

In [28]:
from urllib.parse import quote

def get_semrush_data(url, api_key=os.getenv("SEMRUSH_API_KEY")):
    base_url = "https://api.semrush.com/"
    type_param = "url_organic"
    export_columns = "Ph,Po,Nq,Cp,Co"
    database = "us"
    display_limit = 50
    display_filter = "%2B%7CPo%7CLt%7C50" #Position lower than 50
    display_sort = "po_asc" #Asc sort by position 
       
    full_url = (
        f"{base_url}?type={type_param}&key={api_key}"
        f"&display_limit={display_limit}&export_columns={export_columns}"
        f"&url={url}&database={database}"
        f"&display_filter={display_filter}&display_sort={display_sort}"
    )

    print(full_url)
    
    response = requests.get(full_url)
    if response.status_code == 200:
        api_output = response.content
        print(api_output)
        decoded_output = api_output.decode('utf-8')
        lines = decoded_output.split('\r\n')
        headers = lines[0].split(';')
        json_data = []
        for line in lines[1:]:
            if line:  # Ensure the line is not empty
                values = line.split(';')
                record = {header: value for header, value in zip(headers, values)}
                json_data.append(record)       
        return json_data
    else:
        return []

In [29]:
df_results['SEMRush_Data'] = df_results['Link'].apply(get_semrush_data)

https://api.semrush.com/?type=url_organic&key=cde4f7ee33018efbf83128a179c13a83&display_limit=50&export_columns=Ph,Po,Nq,Cp,Co&url=https://www.reddit.com/r/digitalnomad/comments/vl38d1/which_country_place_in_asia_is_the_best_for/&database=us&display_filter=%2B%7CPo%7CLt%7C50&display_sort=po_asc
b'Keyword;Position;Search Volume;CPC;Competition\r\nbest places to work remotely in asia;1;40;0.00;0.00\r\n'
https://api.semrush.com/?type=url_organic&key=cde4f7ee33018efbf83128a179c13a83&display_limit=50&export_columns=Ph,Po,Nq,Cp,Co&url=https://www.lostcoconuts.com/best-destinations-southeast-asia-digital-nomads/&database=us&display_filter=%2B%7CPo%7CLt%7C50&display_sort=po_asc
b'Keyword;Position;Search Volume;CPC;Competition\r\nbest places to work remotely in asia;6;40;0.00;0.00\r\n'
https://api.semrush.com/?type=url_organic&key=cde4f7ee33018efbf83128a179c13a83&display_limit=50&export_columns=Ph,Po,Nq,Cp,Co&url=https://nomadsembassy.com/best-digital-nomad-cities-in-asia/&database=us&display_fi

In [23]:
test_url = df_results.iloc[0]['Link']
print(test_url)
get_semrush_data(test_url)

https://www.mbopartners.com/state-of-independence/digital-nomads/
https://api.semrush.com/?type=url_organic&key=cde4f7ee33018efbf83128a179c13a83&display_limit=50&export_columns=Ph,Po,Nq,Cp,Co&url=https://www.mbopartners.com/state-of-independence/digital-nomads/&database=us&display_filter=%2B%7CPo%7CLt%7C50&display_sort=po_asc
b'Keyword;Position;Search Volume;CPC;Competition\r\nhow many nomads are there in america;1;1000;0.00;0.00\r\ndigital nomad wanderlust statistics;1;1900;0.00;0.00\r\ndigital nomad statistics;3;30;0.00;0.02\r\ndigital nomad stats;3;30;0.00;0.02\r\ndigital nomad usa;7;40;1.09;0.12\r\ndigitale nomade;8;30;0.00;0.00\r\nnomad for less;8;90;1.78;0.09\r\ndigital nomad companies;10;30;1.01;0.17\r\ndigital nomad jobs 2024;11;30;0.57;0.34\r\ndigital normad;11;30;0.82;0.06\r\ndigital nomand;11;70;0.82;0.06\r\ndigital nomads;11;2900;0.82;0.06\r\ndigital nomadism;12;320;0.82;0.06\r\ndigital nomading;12;590;0.82;0.06\r\ndigital nomad company;13;50;1.01;0.17\r\nnomads work;15;30;

[{'Keyword': 'how many nomads are there in america',
  'Position': '1',
  'Search Volume': '1000',
  'CPC': '0.00',
  'Competition': '0.00'},
 {'Keyword': 'digital nomad wanderlust statistics',
  'Position': '1',
  'Search Volume': '1900',
  'CPC': '0.00',
  'Competition': '0.00'},
 {'Keyword': 'digital nomad statistics',
  'Position': '3',
  'Search Volume': '30',
  'CPC': '0.00',
  'Competition': '0.02'},
 {'Keyword': 'digital nomad stats',
  'Position': '3',
  'Search Volume': '30',
  'CPC': '0.00',
  'Competition': '0.02'},
 {'Keyword': 'digital nomad usa',
  'Position': '7',
  'Search Volume': '40',
  'CPC': '1.09',
  'Competition': '0.12'},
 {'Keyword': 'digitale nomade',
  'Position': '8',
  'Search Volume': '30',
  'CPC': '0.00',
  'Competition': '0.00'},
 {'Keyword': 'nomad for less',
  'Position': '8',
  'Search Volume': '90',
  'CPC': '1.78',
  'Competition': '0.09'},
 {'Keyword': 'digital nomad companies',
  'Position': '10',
  'Search Volume': '30',
  'CPC': '1.01',
  'Com

In [30]:
df_results

,Position,Link,Title,SEMRush_Data
0,1,https://www.reddit.com/r/digitalnomad/comments...,Which country / place in Asia is the best for ...,[{'Keyword': 'best places to work remotely in ...
1,2,https://www.lostcoconuts.com/best-destinations...,12 best destinations in Southeast Asia for dig...,[{'Keyword': 'best places to work remotely in ...
2,3,https://nomadsembassy.com/best-digital-nomad-c...,12 Best Digital Nomad Cities in Asia [2024 Edi...,[{'Keyword': 'best places to work remotely in ...
3,4,https://www.thedigitalnomad.asia/,Digital Nomad Asia,[]
4,5,https://www.goatsontheroad.com/digital-nomad-c...,7 Best Digital Nomad Cities in Asia,[{'Keyword': 'best places to work remotely in ...
5,6,https://www.youtube.com/watch?v=IHUXp49Xlzg,5 INSANELY CHEAP Destinations for Digital Noma...,"[{'Keyword': 'digital nomad spots', 'Position'..."
6,7,https://www.creimermanlaw.com/post/top-southea...,Top Southeast Asian Destinations for Expats an...,[]


## Step 4: Fetch Content
https://jina.ai/reader/

In [31]:
def fetch_content(url):
    print(f'working on {url}')
    headers = {
        'Authorization': f'Bearer {os.getenv("JINA_API_KEY")}',
        'X-Retain-Images': 'none',
        "Accept": "application/json",
        'X-Timeout': '15'
    }

    response = requests.get(f'https://r.jina.ai/{url}', headers=headers).json()
    if response['code'] == 200:
        return response['data']['content']
    elif response['code'] != 200:
        return f"ERROR: {url} blocks Jina API"
    else:
        print(f"ERROR: Failed to use Jina API")


In [32]:
df_results['Content'] = df_results['Link'].apply(fetch_content)

working on https://www.reddit.com/r/digitalnomad/comments/vl38d1/which_country_place_in_asia_is_the_best_for/
working on https://www.lostcoconuts.com/best-destinations-southeast-asia-digital-nomads/
working on https://nomadsembassy.com/best-digital-nomad-cities-in-asia/
working on https://www.thedigitalnomad.asia/
working on https://www.goatsontheroad.com/digital-nomad-cities-asia/
working on https://www.youtube.com/watch?v=IHUXp49Xlzg
working on https://www.creimermanlaw.com/post/top-southeast-asian-destinations-for-expats-and-digital-nomads-in-2024


In [33]:
df_results

,Position,Link,Title,SEMRush_Data,Content
0,1,https://www.reddit.com/r/digitalnomad/comments...,Which country / place in Asia is the best for ...,[{'Keyword': 'best places to work remotely in ...,Which country / place in Asia is the best for ...
1,2,https://www.lostcoconuts.com/best-destinations...,12 best destinations in Southeast Asia for dig...,[{'Keyword': 'best places to work remotely in ...,The destinations in Southeast Asia you should ...
2,3,https://nomadsembassy.com/best-digital-nomad-c...,12 Best Digital Nomad Cities in Asia [2024 Edi...,[{'Keyword': 'best places to work remotely in ...,With a world of excellent digital nomad destin...
3,4,https://www.thedigitalnomad.asia/,Digital Nomad Asia,[],🌴 The Digital Nomad (Asia) - Digital Nomad Asi...
4,5,https://www.goatsontheroad.com/digital-nomad-c...,7 Best Digital Nomad Cities in Asia,[{'Keyword': 'best places to work remotely in ...,While the world adjusts to the new wave of rem...
5,6,https://www.youtube.com/watch?v=IHUXp49Xlzg,5 INSANELY CHEAP Destinations for Digital Noma...,"[{'Keyword': 'digital nomad spots', 'Position'...",5 INSANELY CHEAP Destinations for Digital Noma...
6,7,https://www.creimermanlaw.com/post/top-southea...,Top Southeast Asian Destinations for Expats an...,[],Southeast Asia has emerged as a hotspot for di...


## Step 5: Analyzing Content

## Step 6: Generate Content Plan

## Step 7: Generate Content Draft

## Step 8: Proofread the Draft Post

## Step 9: SEO expert

## Step 10: Summarization